In [6]:
library(extRemes)
library(glue)

In [7]:
# Read data into notebook

infile <- glue("station_data/gev_params_all_stations.csv")
params <- read.csv(file=infile, header=T, sep=',')

In [8]:
# Estimate precipitation depth for the different bounds:

bounds = c('best') # Upper, lower bounds and best estimate

In [9]:
# For each bound and best estimate of the GEV parameters that were estimated in previous
# notebooks, obtain the precipitation depth value from storms of 2-, 5-, 10-, 25-, 50-,
# 100-year. The estimates are estimated in csv files with the bound used (or best estimate),
# in the filename. The output is organized by grid cell (row) and return period (columns).

for (bnd in bounds){
    if (bnd == 'upper'){
        df = subset(params, select = -c(L_loc, loc, L_scale, scale, L_shape,shape))
    } else if (bnd == 'lower'){
        df = subset(params, select = -c(U_loc, loc, U_scale, scale, U_shape,shape) )
    } else if (bnd == 'best'){
        df = subset(params, select = -c(U_loc, L_loc, U_scale, L_scale, U_shape, L_shape))
    }
    params <- t(df)
    model.name <- params["models", ]

    f = (length(model.name))
    rl.cals <- data.frame(matrix(ncol = 7, nrow =length(model.name) ))
    colnames(rl.cals) <- c("models", "2-yr", "5-yr",
                                     "10-yr", "25-yr", "50-yr",
                                     "100-yr")
    rl.cals[1] <- as.vector(model.name)

    for (model in 1:f){

        pmodel <- params[, (model)]
        pmodel <- pmodel[!is.na(pmodel)]
        tryCatch({
        rl.cals.single <- as.vector(rlevd(period=c(2,5,10,25,50,100), 
          loc = as.double(pmodel[3]), scale=as.double(pmodel[4]), shape=as.double(pmodel[5])))
        rl.cals[model, c(2:7)] <- rl.cals.single
        }, error = function(e){})
    }
    rl.cals[is.na(rl.cals)] <- 0
    write.csv(rl.cals, glue('station_data/rp_{bnd}_depth.csv'))
    
}
